경기도 geojson 출처: https://github.com/vuski/admdongkor/blob/master/ver20230701/HangJeongDong_ver20230701.geojson

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/XX401/data/주민등록인구집계현황.csv',encoding='cp949')
df.head()

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2024,5,도,경기도,13654710,966673,1318138,1656250,1887017,2235909,...,643652,785868,901873,1095341,1167475,936383,465821,268688,50208,1497
1,2024,5,시군,경기도 가평군,62389,2694,4259,5610,5491,7241,...,2089,2385,2484,3398,5097,6637,3893,2503,443,18
2,2024,5,읍면동,경기도 가평군 가평읍,21084,1104,1746,2157,2153,2752,...,843,942,1058,1329,1712,1977,1091,803,134,8
3,2024,5,읍면동,경기도 가평군 북면,3608,83,167,189,199,320,...,85,73,83,133,258,504,305,205,44,2
4,2024,5,읍면동,경기도 가평군 상면,5436,112,199,381,365,478,...,87,118,147,220,491,731,428,250,42,0


In [ ]:
df=df[(df['연도']==2024)&(df['월']==5)]
df.head()

,연도,월,행정구역구분명,행정구역명,총 인구수,0~9세,10~19세,20~29세,30~39세,40~49세,...,10~19세 (여),20~29세 (여),30~39세 (여),40~49세 (여),50~59세 (여),60~69세 (여),70~79세 (여),80~89세 (여),90~99세 (여),100세 이상 (여)
0,2024,5,도,경기도,13654710,966673,1318138,1656250,1887017,2235909,...,643652,785868,901873,1095341,1167475,936383,465821,268688,50208,1497
1,2024,5,시군,경기도 가평군,62389,2694,4259,5610,5491,7241,...,2089,2385,2484,3398,5097,6637,3893,2503,443,18
2,2024,5,읍면동,경기도 가평군 가평읍,21084,1104,1746,2157,2153,2752,...,843,942,1058,1329,1712,1977,1091,803,134,8
3,2024,5,읍면동,경기도 가평군 북면,3608,83,167,189,199,320,...,85,73,83,133,258,504,305,205,44,2
4,2024,5,읍면동,경기도 가평군 상면,5436,112,199,381,365,478,...,87,118,147,220,491,731,428,250,42,0


In [ ]:
df=df[['행정구역구분명','행정구역명','총 인구수']]
print(len(df))
print(df.head())

659
  행정구역구분명        행정구역명     총 인구수
0       도        경기도    13654710
1      시군     경기도 가평군      62389
2     읍면동  경기도 가평군 가평읍     21084
3     읍면동   경기도 가평군 북면      3608
4     읍면동   경기도 가평군 상면      5436


In [ ]:
df=df[df['행정구역구분명']=='읍면동']
df

,행정구역구분명,행정구역명,총 인구수
2,읍면동,경기도 가평군 가평읍,21084
3,읍면동,경기도 가평군 북면,3608
4,읍면동,경기도 가평군 상면,5436
5,읍면동,경기도 가평군 설악면,9627
6,읍면동,경기도 가평군 조종면,9210
...,...,...,...
654,읍면동,경기도 화성시 팔탄면,9680
655,읍면동,경기도 화성시 향남읍,86044
656,읍면동,경기도 화성시 화산동,27748
657,읍면동,경기도 화성시동부출장소,0


In [ ]:
import json
import folium

with open('/content/drive/MyDrive/XX401/data/HangJeongDong_ver20230701.geojson') as response:
    geo_json = json.load(response)

# 경기도 시군구 경계 정보를 가진 geo-json 파일 불러오기
# 경기도 행정구역 경계 지리 정보를 사용

geo_json


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# 경기도 지도 만들기
g_map = folium.Map(location=[37.5502, 126.982], zoom_start=9)
g_map

In [41]:
# 총인구수 minmax scaling
from sklearn.preprocessing import MinMaxScaler

mMscaler = MinMaxScaler(feature_range=(0,5))
mMscaler.fit(df[['총 인구수']])

df['총 인구수 scaler'] = mMscaler.transform(df[['총 인구수']])

In [ ]:
import plotly.express as px
fig = px.choropleth_mapbox(data_frame=df, geojson=geo_json,
                           featureidkey="properties.adm_nm",
                           locations=df['행정구역명'], color="총 인구수 scaler",
                           opacity=0.5, zoom=9,
                           center={"lat": 37.541, "lon": 126.986},
                           mapbox_style="carto-positron"
                           )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
#fig.write_html("index.html")